# Documentation for the Code Challenges using TiddlyWiki

The notebook aims to convert code solutions into comprehensive documentation stored on a TiddlyWiki-powered webpage. Each solution becomes a dedicated 'tiddler,' providing self-contained implementation details. The seamless integration of these tiddlers enriches TiddlyWiki, offering readers a valuable resource to explore and learn solution concepts conveniently. The TiddlyWiki-based documentation becomes an accessible and informative reference for enthusiasts and learners alike.

**The link to the final wiki containing the documentation can be found [here](https://ai-code-challenge.tiddlyhost.com/).**

![A clever cat stands attentively at the library, engrossed in reading through code documentation.](./code_illustration/tiddlywiki_docs.jpg)

# Leveraging TiddlyWiki for Enhanced Documentation

This project utilizes TiddlyWiki, a unique non-linear note-taking and personal wiki software. It enables efficient information organization in an interconnected and adaptable manner. Unlike traditional wikis, TiddlyWiki operates as a self-contained application within a web browser, using a single HTML file with JavaScript code.

The project leverages TiddlyWiki's flexibility to document code generated in the notebook "Automated Solutions for the Challenges." By doing so, it provides a powerful tool for effective information organization and knowledge management support.

## Accessing the Encrypted File to Obtain the API Ke

In [ ]:
import os
import pandas as pd
import openai
import getpass
import base64
import time
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

In [ ]:
def generate_salt():
    return os.urandom(16)

def generate_fernet_key(password, salt):
    password = password.encode()
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password))
    return key

def get_decrypted_api_key(password):
    try:
        with open("api_key.enc", "rb") as file:
            encrypted_api_key = file.read()

        with open("salt.bin", "rb") as file:
            salt = file.read()

        # Derive the Fernet key from the password and base64-decode the salt
        key = generate_fernet_key(password, salt)

        cipher_suite = Fernet(key)
        decrypted_api_key = cipher_suite.decrypt(encrypted_api_key).decode()

        return decrypted_api_key
    except FileNotFoundError:
        print("API key file not found.")
        return None
    except Exception as e:
        print("Error occurred while decrypting the API key:", str(e))
        return None

# Prompt user for the password again to decrypt the API key
password_for_decryption = getpass.getpass("Enter your password to decrypt the API key: ")

# Retrieve and decrypt the API key
openai.api_key = get_decrypted_api_key(password_for_decryption)

## Creating TiddlyWiki Files with Challenge Descriptions

In [ ]:
df = pd.read_csv('code_challenge.csv', delimiter=';', encoding='latin') 

# Drop index column
df = df.reset_index(drop=True) 

beginner_df = df.loc[df['difficulty'] == 'Beginner']
beginner_df.head()

In [ ]:
def generate_problem_enunciation(problem_parameter):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"Elaborate what is {problem_parameter}  describing it as a problem enunciation.",
        max_tokens=333,
        n=1
    )
    return response.choices[0].text.strip()

def generate_challenge_tiddlers(tiddlywiki_folder, dataframe):
    
    programming_languages = ['Python', 'JavaScript', 'Java', 'C#', 'C++']
    
    if not os.path.exists(tiddlywiki_folder):
        os.makedirs(tiddlywiki_folder)

    for index, row in dataframe.iterrows():
        problem_id = '{:03d}'.format(row['id'])\n",
        problem_title = row['name']
        difficulty = row['difficulty']
        category = row['category']
        problem_parameter = f' {problem_title} ({category})'
        problem_enunciation = generate_problem_enunciation(problem_parameter)
        
        tiddler_title = f'{problem_id} {problem_title}'
        tiddler_header = f"tags: [[{difficulty}]] [[{category}]] \ntitle: {tiddler_title}\n\n"
        tiddler_enunciation = f"{problem_enunciation}\n\n"
        tiddler_solution = '__Code Solutions__\n\n'
        for language in programming_languages:
            tiddler_solution = tiddler_solution + f"\n* [[{tiddler_title} - {language}]]"
        tiddler_content = tiddler_header + tiddler_enunciation + tiddler_solution
        
        
        tiddler_filename = f"{tiddler_title}.tid" 
        tiddler_path = os.path.join(tiddlywiki_folder, tiddler_filename)

        with open(tiddler_path, 'w') as tiddler_file:
            tiddler_file.write(tiddler_content)
        time.sleep(23)

    print("TiddlyWiki tiddler files generated successfully!")

current_path = os.getcwd()
tiddlywiki_folder = os.path.join(current_path, "code_challenge_documentation", "tiddlywiki")
generate_challenge_tiddlers(tiddlywiki_folder, beginner_df)


## Transforming Python Code into TiddlyWiki Files

In [ ]:
def generate_tiddlers(python_folder_path, tiddlywiki_folder):
    
    if not os.path.exists(tiddlywiki_folder):
        os.makedirs(tiddlywiki_folder)
    
    for filename in os.listdir(python_folder_path):
        if filename.endswith(".py"):
            with open(os.path.join(python_folder_path, filename), 'r') as python_file:
                content = python_file.read()
            tiddler_title = f'title:{filename} - Python\n'.replace('.py', '')
            tiddler_tag = "tags: Python\n"
            content = content.replace("```python", '').replace("```", '')
            tiddler_content = f"{tiddler_title}{tiddler_tag}\n```Python\n{content}\n```"
            tiddler_filename = os.path.splitext(filename)[0] + "- Python.tid"
            tiddler_path = os.path.join(tiddlywiki_folder, tiddler_filename)
            
            with open(tiddler_path, 'w') as tiddler_file:
                tiddler_file.write(tiddler_content)
    print("TiddlyWiki tiddler files generated successfully!")

current_path = os.getcwd()
python_folder_path = os.path.join(current_path, "code_challenge_solution", "python")
tiddlywiki_folder = os.path.join(current_path, "code_challenge_documentation", "python")
generate_tiddlers(python_folder_path, tiddlywiki_folder)


## Transforming JavaScript Code into TiddlyWiki Files

In [ ]:
def generate_tiddlers(python_folder_path, tiddlywiki_folder):
    
    if not os.path.exists(tiddlywiki_folder):
        os.makedirs(tiddlywiki_folder)
    
    for filename in os.listdir(python_folder_path):
        if filename.endswith(".js"):
            with open(os.path.join(python_folder_path, filename), 'r') as python_file:
                content = python_file.read()
            tiddler_title = f'title:{filename} - JavaScript\n'.replace('.js', '')
            content = content.replace("```javascript", '').replace("```", '')
            tiddler_tag = "tags: JavaScript\n"
            tiddler_content = f"{tiddler_title}{tiddler_tag}\n```JavaScript\n{content}\n```"
            tiddler_filename = os.path.splitext(filename)[0] + "- JavaScript.tid"
            tiddler_path = os.path.join(tiddlywiki_folder, tiddler_filename)
            
            with open(tiddler_path, 'w') as tiddler_file:
                tiddler_file.write(tiddler_content)
    print("TiddlyWiki tiddler files generated successfully!")

current_path = os.getcwd()
python_folder_path = os.path.join(current_path, "code_challenge_solution", "javascript")
tiddlywiki_folder = os.path.join(current_path, "code_challenge_documentation", "javascript")
generate_tiddlers(python_folder_path, tiddlywiki_folder)


## Transforming Java Code into TiddlyWiki Files

## Transforming C# Code into TiddlyWiki Files

## Transforming C++ Code into TiddlyWiki Files